##### Copyright 2026 Google LLC.

# Engineering Reliability (API Features)

To build a production-ready robotics system, you need more than just smart reasoning—you need **determinism and reliability**.

A robot controller cannot parse "chatty" or inconsistent text. It requires strict data formats and predictable latency. This notebook explores the Gemini API features that help bridge the gap between probabilistic AI and deterministic hardware.

This notebook covers:
1.  **JSON Mode:** Enforcing type safety to prevent controller crashes.
2.  **Thinking Budgets:** Controlling the trade-off between response speed and reasoning depth.
3.  **Code Execution:** Using a Python sandbox for precise math and high-resolution inspections.

## 1. Setup & Initialization

In [ ]:
%pip install -U -q google-genai google-colab

In [ ]:
import os
import urllib.request
import textwrap
import time
from PIL import Image

if not os.path.exists('robotics_utils.py'):
    url = "https://raw.githubusercontent.com/williamito/robotics-samples/refs/heads/main/robotics_utils.py"
    urllib.request.urlretrieve(url, 'robotics_utils.py')

import robotics_utils as utils
utils.download_assets()

In [ ]:
from google.colab import userdata
from google import genai
from google.genai import types

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)
MODEL_ID = "gemini-3-flash-preview"

## 2. Part 1: Determinism (JSON Mode)

### Example: Eliminating Filler Text
By enforcing JSON mode, we create a strict contract between the AI and our robot controller, ensuring the output is always machine-readable.

In [ ]:
prompt = "List 3 common household tasks for a robot."

# Enforce JSON mode via configuration
response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(response_mime_type="application/json")
)
print(response.text)

[
  "Vacuuming floors",
  "Mopping surfaces",
  "Folding laundry"
]


## 3. Part 2: Latency Control (Thinking Budget)

### Example: Speed vs. Depth
With **Thinking**, the model can take more time to reason about complex queries. For time-critical robotics loops (e.g., visual servoing), you can set a low budget. For complex planning, you can increase it.

*Note: Thinking is supported on select models like `gemini-3-flash-preview`.*

In [ ]:
prompt = "Define 'Inverse Kinematics' in one sentence."

# Scenario A: Low Latency (Fast Mode)
t0 = time.time()
client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0)
    )
)
print(f"Fast Mode Time: {time.time() - t0:.2f}s")

# Scenario B: Reasoning (Dynamic Mode)
t1 = time.time()
client.models.generate_content(model=MODEL_ID, contents=prompt)
print(f"Dynamic Mode Time: {time.time() - t1:.2f}s")

Fast Mode Time: 0.94s
Dynamic Mode Time: 2.83s


## 4. Part 3: Precise Action (Code Execution)

### Example: Sensor Reading with Zoom
The model can write Python code to crop and zoom into high-resolution images, mimicking a robot moving its head closer for a better look.

In [ ]:
img_sensor = Image.open("air_quality.jpeg")
prompt_zoom = "What is the air quality reading? Use code execution to zoom in on the display first."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[img_sensor, prompt_zoom],
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    )
)

for part in response.candidates[0].content.parts:
    if part.text: print(f"Model Answer: {part.text}")

Model Answer: Based on the zoomed-in image of the display, the PM2.5 air quality reading is **014**. According to the sticker below the display, a reading between 0-12 particles is considered "Excellent" and 13-35 is "Moderate," placing this reading at the low end of the "Moderate" range.


## Conclusion

You've completed the core capabilities tour!

**Where to go next:**
*   **[Gemini API Documentation](https://ai.google.dev/gemini-api/docs/):** Deep dive into the API.